# Soilgrids Analysis
## Water Storage Capacity Analysis for Bachelor Thesis

This notebook contains the main code for analyzing water storage capacity using soilgrids data.

## 1. Import Libraries

In [8]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soilgrids
from soilgrids import SoilGrids
import rasterio
import xarray as xr
import pyproj

# Add more libraries as needed for soilgrids analysis

In [2]:
#initialize SoilGrids 
sg = SoilGrids()


In [5]:
sg.get_coverage_list("sand")

"sand" map service includes 30 coverages(maps)

sand_0-5cm_Q0.05
sand_0-5cm_Q0.5
sand_0-5cm_Q0.95
sand_0-5cm_mean
sand_0-5cm_uncertainty
sand_5-15cm_Q0.5
sand_5-15cm_Q0.05
sand_5-15cm_Q0.95
sand_5-15cm_mean
sand_5-15cm_uncertainty
sand_15-30cm_Q0.5
sand_15-30cm_Q0.05
sand_15-30cm_Q0.95
sand_15-30cm_mean
sand_15-30cm_uncertainty
sand_30-60cm_Q0.05
sand_30-60cm_Q0.5
sand_30-60cm_Q0.95
sand_30-60cm_mean
sand_30-60cm_uncertainty
sand_60-100cm_Q0.5
sand_60-100cm_Q0.05
sand_60-100cm_Q0.95
sand_60-100cm_mean
sand_60-100cm_uncertainty
sand_100-200cm_Q0.05
sand_100-200cm_Q0.5
sand_100-200cm_Q0.95
sand_100-200cm_mean
sand_100-200cm_uncertainty


In [ ]:
#TODO: 
# get bounding box
# get data from bounding box
# get formula for litterally anything just to test it out
# apply data to formula
# visualize data

# required variables 
- sand  # sand content [%]
- clay  # clay content [%]
- soc   # soil organic carbon [g kg-1]
- bdod    # bulk density [kg m-3]

In [9]:
def calculate_dimensions(west, south, east, north, crs, pixel_size_m=250):
    """
    Calculate width and height in pixels based on bounding box and desired pixel size.

    Parameters:
    -----------
    west, south, east, north : float
        Bounding box coordinates in the specified CRS
    crs : str
        Coordinate reference system (e.g., "EPSG:4326")
    pixel_size_m : float, optional
        Desired pixel size in meters (default: 250)

    Returns:
    --------
    tuple : (width, height) in pixels
    """
    # Create transformer to convert from lat/lon to meters
    # Use a local UTM or equidistant projection for accurate distance calculation

    # Get the center point to determine appropriate UTM zone
    center_lon = (west + east) / 2
    center_lat = (south + north) / 2

    # Create a local equal-area projection centered on the AOI
    proj_string = f"+proj=aeqd +lat_0={center_lat} +lon_0={center_lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

    # Create coordinate transformer
    transformer = pyproj.Transformer.from_crs(
        crs,
        proj_string,
        always_xy=True
    )

    # Transform corner coordinates to meters
    west_m, south_m = transformer.transform(west, south)
    east_m, north_m = transformer.transform(east, north)

    # Calculate dimensions in meters
    width_m = east_m - west_m
    height_m = north_m - south_m

    # Calculate dimensions in pixels
    width_px = int(abs(width_m) / pixel_size_m)
    height_px = int(abs(height_m) / pixel_size_m)

    return width_px, height_px

In [10]:
# define the crs and bounding box
crs = "urn:ogc:def:crs:EPSG::4326"
west, south, east, north = [-79.30, -2.87, -79.18, -2.77]
width, height = calculate_dimensions(west, south, east, north, crs, pixel_size_m=250)
print(f"Width: {width} pixels")
print(f"Height: {height} pixels")

Width: 53 pixels
Height: 44 pixels


In [ ]:
#option 2: use resolutions instead of dimensions
import math
lat = (south + north) / 2
meters_per_degree_lon = 111320 * math.cos(math.radians(abs(lat)))
meters_per_degree_lat = 111320

resx = 250 / meters_per_degree_lon
resy = 250 / meters_per_degree_lat

In [11]:
#download the silt content
silt = sg.get_coverage_data(service_id="silt",
                                    coverage_id='silt_0-5cm_mean',
                                    crs=crs,
                                    west=west,
                                    south=south,
                                    east=east,
                                    north=north,
                                    #resx=resx,
                                    #resy=resy,
                                    width=width,
                                    height=height,
                                    output='test_silt.tif')
#print the first couple of lines
print(silt.head())

# show metadata
for key, value in silt.attrs.items():
    print(f'{key}: {value}')


silt_masked = silt.where(silt != -32768) #value for nodata from soilgrids

# Or use the _FillValue attribute
nodata = silt.attrs.get('_FillValue', -32768)
silt_masked = silt.where(silt != nodata)

# Now get proper statistics
print("\nData summary (excluding NoData):")
print(f"Min: {silt_masked.min().values}")
print(f"Max: {silt_masked.max().values}")
print(f"Mean: {silt_masked.mean().values}")
print(f"Count of valid pixels: {silt_masked.count().values}")
print(f"Count of NoData pixels: {(silt == nodata).sum().values}")


<xarray.DataArray (band: 1, y: 5, x: 5)> Size: 50B
[25 values with dtype=int16]
Coordinates:
  * band         (band) int32 4B 1
  * y            (y) float64 40B -2.771 -2.773 -2.776 -2.778 -2.78
  * x            (x) float64 40B -79.3 -79.3 -79.29 -79.29 -79.29
    spatial_ref  int32 4B 0
Attributes:
    TIFFTAG_XRESOLUTION:     72
    TIFFTAG_YRESOLUTION:     72
    TIFFTAG_RESOLUTIONUNIT:  2 (pixels/inch)
    AREA_OR_POINT:           Area
    _FillValue:              -32768
    scale_factor:            1.0
    add_offset:              0.0
TIFFTAG_XRESOLUTION: 72
TIFFTAG_YRESOLUTION: 72
TIFFTAG_RESOLUTIONUNIT: 2 (pixels/inch)
AREA_OR_POINT: Area
_FillValue: -32768
scale_factor: 1.0
add_offset: 0.0

Data summary (excluding NoData):
Min: 323.0
Max: 429.0
Mean: 386.1640319824219
Count of valid pixels: 2280
Count of NoData pixels: 52


## Download Soil Properties Data (0-5cm depth)
Download all required soil properties from SoilGrids for the specified bounding box.

In [12]:
# Define the soil properties to download
soil_properties = {
    'sand': 'sand_0-5cm_mean',
    'clay': 'clay_0-5cm_mean',
    'silt': 'silt_0-5cm_mean',
    'soc': 'soc_0-5cm_mean',
    'bdod': 'bdod_0-5cm_mean'
}

# Dictionary to store the downloaded data
soil_data_0_5cm = {}

# Download each soil property
for property_name, coverage_id in soil_properties.items():
    print(f"\nDownloading {property_name}...")
    
    data = sg.get_coverage_data(
        service_id=property_name,
        coverage_id=coverage_id,
        crs=crs,
        west=west,
        south=south,
        east=east,
        north=north,
        width=width,
        height=height,
        output=f'output_data/{property_name}_0-5cm.tif'
    )
    
    # Mask NoData values
    nodata = data.attrs.get('_FillValue', -32768)
    data_masked = data.where(data != nodata)
    
    # Store in dictionary
    soil_data_0_5cm[property_name] = data_masked
    
    # Print basic statistics
    print(f"{property_name} - Min: {data_masked.min().values:.2f}, "
          f"Max: {data_masked.max().values:.2f}, "
          f"Mean: {data_masked.mean().values:.2f}")

print("\n✓ All soil properties downloaded successfully!")


sand - Min: 372.00, Max: 484.00, Mean: 422.96

clay - Min: 141.00, Max: 248.00, Mean: 190.88

silt - Min: 323.00, Max: 429.00, Mean: 386.16

soc - Min: 558.00, Max: 1931.00, Mean: 1215.48

bdod - Min: 92.00, Max: 118.00, Mean: 104.99

✓ All soil properties downloaded successfully!


In [13]:
# Access individual variables for convenience
sand_0_5cm = soil_data_0_5cm['sand']
clay_0_5cm = soil_data_0_5cm['clay']
silt_0_5cm = soil_data_0_5cm['silt']
soc_0_5cm = soil_data_0_5cm['soc']
bdod_0_5cm = soil_data_0_5cm['bdod']

print("Variables created:")
print("- sand_0_5cm: Sand content [g/kg]")
print("- clay_0_5cm: Clay content [g/kg]")
print("- silt_0_5cm: Silt content [g/kg]")
print("- soc_0_5cm: Soil organic carbon [dg/kg]")
print("- bdod_0_5cm: Bulk density [cg/cm³]")

Variables created:
- sand_0_5cm: Sand content [g/kg]
- clay_0_5cm: Clay content [g/kg]
- silt_0_5cm: Silt content [g/kg]
- soc_0_5cm: Soil organic carbon [dg/kg]
- bdod_0_5cm: Bulk density [cg/cm³]


## 2. Load Input Data

In [ ]:
# Load data from input_data folder
# Example:
# data = pd.read_csv('input_data/your_data_file.csv')

## 3. Data Processing and Analysis

In [ ]:
# Add your analysis code here

## 4. Visualization

In [ ]:
# Create visualizations
# Example:
# plt.figure(figsize=(10, 6))
# plt.plot(data)
# plt.title('Water Storage Capacity Analysis')
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.show()

## 5. Save Results

In [ ]:
# Save results to output_data folder
# Example:
# results.to_csv('output_data/analysis_results.csv', index=False)